# Getting started

## Installation

JVol can be installed using [Pip Installs Packages (PIP)](https://pip.pypa.io/):

```shell
pip install git+git://github.com/fepegar/jvol.git
```

Let's install it in this notebook.

In [ ]:
%%capture
%pip install git+git://github.com/fepegar/jvol.git


## Usage

Let's first download some NIfTI data from the [Colin 27 Average Brain 2008](https://nist.mni.mcgill.ca/colin-27-average-brain-2008/) MNI template.

In [ ]:
%%bash
curl -O -s https://packages.bic.mni.mcgill.ca/mni-models/colin27/mni_colin27_2008_nifti.zip
unzip -q -o mni_colin27_2008_nifti.zip
ls -lh *.nii


We'll use the `jvol` command to encode an existing image.
Use `jvol --help` to get a full list of features of the CLI tool.

In [ ]:
!jvol --help


Let's encode an image with the default settings.

In [ ]:
!jvol colin27_t1_tal_hires.nii colin27_t1_tal_hires.jvol


Let's use Python to check how we've done.
We'll also install [TorchIO](https://torchio.readthedocs.io/) to read some of the image metadata.

In [ ]:
%%capture
%pip install "torchio[plot]"


In [ ]:
from pathlib import Path
import numpy as np
import torchio as tio
from humanize import naturalsize

def compare_sizes(original_path: Path, compressed_path: Path) -> None:
    def get_num_pixels(path):
        return np.prod(tio.ScalarImage(path).shape)
    num_pixels = get_num_pixels(original_path)
    original_size = original_path.stat().st_size
    original_bits_per_pixel = 8 * original_size / num_pixels
    compressed_size = compressed_path.stat().st_size
    compressed_bits_per_pixel = 8 * compressed_size / num_pixels

    compression_ratio = original_size / compressed_size
    print(f'Original file size: {naturalsize(original_size)} ({original_bits_per_pixel:.2f} bits/pixel)')
    print(f'Compressed file size: {naturalsize(compressed_size)} ({compressed_bits_per_pixel:.2f} bits/pixel)')
    print(f'Compression ratio: {compression_ratio:.1f}×')

nii_path = Path('colin27_t1_tal_hires.nii')
jvol_path = Path('colin27_t1_tal_hires.jvol')

compare_sizes(nii_path, jvol_path)


Not bad! Let's compare with lossless compression.

In [ ]:
!gzip -k -f colin27_t1_tal_hires.nii


In [ ]:
nii_gz_path = Path('colin27_t1_tal_hires.nii.gz')

compare_sizes(nii_path, nii_gz_path)


It seems that we're doing much better!

You might be wondering about the visual quality of the compressed image, compared to the original.
Let's use [TorchIO](https://torchio.readthedocs.io/) to visualize both images.

We'll need to convert the compressed image to a format that TorchIO can read.
We can use `jvol` to decode the image back to NIfTI.

In [ ]:
!jvol colin27_t1_tal_hires.jvol colin27_t1_tal_hires_from_jvol.nii


In [ ]:
import torchio as tio

def compare_images(original_path: Path, compressed_path: Path) -> None:
    original = tio.ScalarImage(original_path)
    compressed = tio.ScalarImage(compressed_path)
    both = tio.Subject({'Original': original, 'Compressed': compressed})
    both.plot(figsize=(10, 6))

from_jvol_path = Path('colin27_t1_tal_hires_from_jvol.nii')
compare_images(nii_path, from_jvol_path)


They look quite similar, so it seems that this amount of compression might be acceptable for certain applications.
Can we push the compression ratio even further?
Let's use a smaller quality setting.

In [ ]:
!jvol colin27_t1_tal_hires.nii colin27_t1_tal_hires_low_quality.jvol --quality 5


Again, let's convert back so TorchIO can read it.

In [ ]:
!jvol colin27_t1_tal_hires_low_quality.jvol colin27_t1_tal_hires_low_quality.nii


In [ ]:
from_jvol_tiny_path = Path('colin27_t1_tal_hires_low_quality.nii')
compare_images(nii_path, from_jvol_tiny_path)


The artifacts are slightly noticeable this time, but the image is visually still very similar.
Let's see how much we've compressed the original image.

In [ ]:
jvol_tiny_path = Path('colin27_t1_tal_hires_low_quality.jvol')
compare_sizes(nii_path, jvol_tiny_path)
